# Requirements for this program to run.
* Install the following via Anaconda Prompt
    - pyqt5 (pip install pyqt5)
    - pyqt5-tools (pip install pyqt5-tools)
    - tensorflow
    - imageai
* Trianed Model and .json file (Both must be located as the same path of this file)
* Qt Designer
* Pandas

In [1]:
# * Install the following via Anaconda Prompt
#-------------------------------------------------
# pip install opencv-python==4.1.2.30
# pip uninstall keras-nightly -y
# pip install keras==2.3.1
# pip install tensorflow==1.14.0
# pip install 'h5py==2.10.0' --force-reinstall
# pip install imageai==2.1.5
# pip install pyqt5
# conda install -c anaconda pandas

## Detection of Cigarette Smokers in Public Places Code

In [1]:
#Importing Libraries
#-------------------------------------------------
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtWidgets import QApplication, QMainWindow, QDesktopWidget, QFileDialog, QMessageBox
from imageai.Detection.Custom import CustomObjectDetection
from imageai.Detection.Custom import CustomVideoObjectDetection
import os
import cv2
import sys
import csv 
import time
import csv
import argparse 
import datetime 
import pandas as pd
#-------------------------------------------------

#Application's Details
#-------------------------------------------------
class MyWindow(QMainWindow):
    def __init__(self):
        super(MyWindow, self).__init__()
        #Set Fixed Resolution Size to 600
        self.setFixedSize(800, 600)
        self.setWindowTitle("Detection of Cigarette Smokers in Public Places")
        #Background Color
        self.setStyleSheet("background-color: rgb(145, 148, 148)")
        self.initUI()
        self.center()
#-------------------------------------------------
        
#Set Form to Center Screen
#-------------------------------------------------
    def center(self):
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())
#-------------------------------------------------
    
#Code for widgets (Labels and Buttons)
#-------------------------------------------------
    def initUI(self):
            #Icon
            icon = QtGui.QIcon()
            icon.addPixmap(QtGui.QPixmap("art/student.ico"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.setWindowIcon(icon)
            self.setWindowOpacity(30.0)
            self.setToolButtonStyle(QtCore.Qt.ToolButtonTextBesideIcon)
            self.setDocumentMode(False)
            
            #Title Label
            self.title = QtWidgets.QLabel(self)
            self.title.setText("Detection of Cigarette Smokers")    
            self.title.setFont(QtGui.QFont('Source Sans Pro', 16))
            self.title.setStyleSheet("font-weight: bold");
            self.title.setAlignment(QtCore.Qt.AlignCenter)
            self.title.setGeometry(QtCore.QRect(0, 35, 801, 41))
            
            #Title 2
            self.title2 = QtWidgets.QLabel(self)
            self.title2.setText("Using Deep Learning")    
            self.title2.setFont(QtGui.QFont('Source Sans Pro', 16))
            self.title2.setStyleSheet("font-weight: bold");
            self.title2.setAlignment(QtCore.Qt.AlignCenter)
            self.title2.setGeometry(QtCore.QRect(0, 75, 801, 41))
            
            #BSU Icon
            self.iconlabel = QtWidgets.QLabel(self)
            self.iconlabel.setText("")
            self.iconlabel.setPixmap(QtGui.QPixmap("art/bsu.png"))
            self.iconlabel.setScaledContents(True)
            self.iconlabel.setGeometry(QtCore.QRect(50, 40, 80, 80))
            
            #Live Label
            self.livelabel = QtWidgets.QLabel(self)
            self.livelabel.setText("WEBCAM")
            self.livelabel.setFont(QtGui.QFont('Berlin Sans FB', 16))
            self.livelabel.setAlignment(QtCore.Qt.AlignCenter)
            self.livelabel.setGeometry(QtCore.QRect(100, 210, 111, 41))
            
            #Live Button
            self.livebtn = QtWidgets.QPushButton(self)
            self.livebtn.setGeometry(QtCore.QRect(70, 250, 165, 165))
            self.livebtn.setText("")
            
            #Live Icon
            livebtnicon = QtGui.QIcon()
            livebtnicon.addPixmap(QtGui.QPixmap("art/webcam.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.livebtn.setIcon(livebtnicon)
            self.livebtn.setIconSize(QtCore.QSize(160, 160))
            self.livebtn.setFlat(True) 
            
            #Button Click Event
            self.livebtn.clicked.connect(self.Livestream)
            
            #Import Live Label
            self.implivelabel = QtWidgets.QLabel(self)
            self.implivelabel.setText("Use Live Feed")
            self.implivelabel.setFont(QtGui.QFont('Berlin Sans FB', 9))
            self.implivelabel.setAlignment(QtCore.Qt.AlignCenter)
            self.implivelabel.setGeometry(QtCore.QRect(75, 420, 161, 31))
            
            #Image Label
            self.Imglabel = QtWidgets.QLabel(self)
            self.Imglabel.setText("IMAGE")    
            self.Imglabel.setFont(QtGui.QFont('Berlin Sans FB', 16))
            self.Imglabel.setAlignment(QtCore.Qt.AlignCenter)
            self.Imglabel.setGeometry(QtCore.QRect(315, 200, 171, 51))
            
            #Image Button
            self.ImgBtn = QtWidgets.QPushButton(self)
            self.ImgBtn.setGeometry(QtCore.QRect(320, 250, 165, 165))
            self.ImgBtn.setText("")
            
            #Image Icon
            imgbtnicon = QtGui.QIcon()
            imgbtnicon.addPixmap(QtGui.QPixmap("art/image.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.ImgBtn.setIcon(imgbtnicon)
            self.ImgBtn.setIconSize(QtCore.QSize(160, 160))
            self.ImgBtn.setFlat(True)
            
            #Image Button Click Event
            self.ImgBtn.clicked.connect(self.getImage)
            
            #Import Image Label
            self.impimglabel = QtWidgets.QLabel(self)
            self.impimglabel.setText("Import Image")
            self.impimglabel.setFont(QtGui.QFont('Berlin Sans FB', 9))
            self.impimglabel.setAlignment(QtCore.Qt.AlignCenter)
            self.impimglabel.setGeometry(QtCore.QRect(340, 420, 131, 31))
            
            #Video Label
            self.Vidlabel = QtWidgets.QLabel(self)
            self.Vidlabel.setText("VIDEO")
            self.Vidlabel.setFont(QtGui.QFont('Berlin Sans FB', 16))
            self.Vidlabel.setAlignment(QtCore.Qt.AlignCenter)
            self.Vidlabel.setGeometry(QtCore.QRect(590, 200, 101, 51))
            
            #Video Button
            self.VidBtn = QtWidgets.QPushButton(self)
            self.VidBtn.setGeometry(QtCore.QRect(560, 250, 165, 165))
            self.VidBtn.setText("")
            
            #Video Icon
            vidbtnicon = QtGui.QIcon()
            vidbtnicon.addPixmap(QtGui.QPixmap("art/video.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
            self.VidBtn.setIcon(vidbtnicon)
            self.VidBtn.setIconSize(QtCore.QSize(160, 160))
            self.VidBtn.setFlat(True)                        
            
            #Button Click Event
            self.VidBtn.clicked.connect(self.getVideo)
            
            #Import Video Label
            self.impvidlabel = QtWidgets.QLabel(self)
            self.impvidlabel.setText("Import Video")
            self.impvidlabel.setFont(QtGui.QFont('Berlin Sans FB', 9))
            self.impvidlabel.setAlignment(QtCore.Qt.AlignCenter)
            self.impvidlabel.setGeometry(QtCore.QRect(565, 420, 161, 31))
#-------------------------------------------------

#Select Image from your Computer to Detect
#-------------------------------------------------
    def getImage(self):
        fname = QFileDialog.getOpenFileName(self, 'Open file', 'c:/',"Image files (*.jpg *.png)")
        #Setting Variable name "imagePath" to global so it can be called at def DetectImage()
        global imagePath
        imagePath = fname[0]
        self.DetectImage()
        
#Code For Single Image Detection
#-------------------------------------------------
    def DetectImage(self):
        from datetime import datetime
        today = datetime.now()
        #Main Code for Hololens
        detector = CustomObjectDetection()
        detector.setModelTypeAsYOLOv3()
        #Model Used
        detector.setModelPath("detection_model-ex-005--loss-0003.607.h5")
        detector.setJsonPath("detection_config.json")
        detector.loadModel()
        detections = detector.detectObjectsFromImage(input_image=imagePath, output_image_path= imagePath + "detected_cigarettes.jpg", minimum_percentage_probability=30)
        
    
        #Create CSV File from the image
        with open('single-image.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            #columns name
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (eachObject) in (detections):
                writer.writerow([eachObject["name"], eachObject["percentage_probability"]])
        self.popupImage()

#Show popup Window after Finishing the Image Detection
#-------------------------------------------------
    def popupImage(self):
        msg = QMessageBox()
        msg.setWindowTitle("Single-Image Detection")
        msg.setText("Single Detection Finish")
        msg.setIcon(QMessageBox.Information)
        x = msg.exec_()


#Select Video from your Computer to Detect
#-------------------------------------------------
    def getVideo(self):
        vidname = QFileDialog.getOpenFileName(self, 'Open file', 'c:/',"Video files (*.avi *.mp4)")
        #Setting Variable name "aviPath" to global so it can be called at def DetectImage()
        global aviPath
        aviPath = vidname[0]
        #call def DectectVideo
        self.DetectVideo()
        
#Code for Video Detection
#-------------------------------------------------
    def DetectVideo(self):
        from datetime import datetime
        classid = []
        percentage =[]
        execution_path = os.getcwd()

    # FOR SMOKING DETECTION
        def forFrame(frame_number, output_array, output_count):
            for eachObject in output_array:
                classid.append(eachObject["name"])
                percentage.append(eachObject["percentage_probability"])
                
        rec_detector = CustomVideoObjectDetection()
        rec_detector.setModelTypeAsYOLOv3()
        rec_detector.setModelPath("detection_model-ex-005--loss-0003.607.h5")
        rec_detector.setJsonPath("detection_config.json")
        rec_detector.loadModel()
        rec_detector.detectObjectsFromVideo(input_file_path=aviPath,
                                                  output_file_path=os.path.join(execution_path, aviPath + "detected_cigarettes"),
                                                  per_frame_function=forFrame,
                                                  minimum_percentage_probability=30,
                                                  log_progress=True)
        
        
        #Create CSV File from the image
        with open('video-image.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            #columns name
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (classid_x, percentage_x) in zip (classid, percentage):
                writer.writerow([classid_x, percentage_x])
        self.popupVid()
        
        
#Show popup Window after Finishing the Video Detection
#-------------------------------------------------
    def popupVid(self):
        msg2 = QMessageBox()
        msg2.setWindowTitle("Video Detection")
        msg2.setText("Video Dectection Finish")
        msg2.setIcon(QMessageBox.Information)
        y = msg2.exec_()
        
#Code for Live Detection
#-------------------------------------------------
    def Livestream(self):
        from datetime import datetime
        classid = []
        percentage =[]

        execution_path = os.getcwd()
        
        camera = cv2.VideoCapture(0)
        
    # FOR SMOKING DETECTION
        #Get Values
        def forFrame(frame_number, output_array, output_count):
            for eachObject in output_array:
                classid.append(eachObject["name"])
                percentage.append(eachObject["percentage_probability"])
                
        video_detector = CustomVideoObjectDetection()
        video_detector.setModelTypeAsYOLOv3()
        #Model Used
        video_detector.setModelPath("detection_model-ex-005--loss-0003.607.h5")
        video_detector.setJsonPath("detection_config.json")
        video_detector.loadModel()
        #Timer is Set to 3 Seconds, to change, change dection_timeout value to desired seconds.
        video_detector.detectObjectsFromVideo(camera_input=camera, 
                                                            output_file_path=os.path.join(execution_path, "LiveFeedOutput"),
                                                            per_frame_function=forFrame,  minimum_percentage_probability=50,
                                                            detection_timeout=3)

        
        #Create CSV File from the image
        with open('live_data.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            #columns name
            writer.writerow(['CLASS', 'PERCENTAGE'])
            for (classid_x, percentage_x) in zip (classid, percentage):
                writer.writerow([classid_x, percentage_x])
        self.popupLive()
        
        
#Show popup Window after Finishing the Live Detection
#-------------------------------------------------
    def popupLive(self):
        msg3 = QMessageBox()
        msg3.setWindowTitle("Live Detection")
        msg3.setText("Live Dectection Finish")
        msg3.setIcon(QMessageBox.Information)
        z = msg3.exec_()
        
#Open Form
if __name__ == "__main__":
    app = QApplication(sys.argv)
    win = MyWindow()
    win.show()
    sys.exit(app.exec_())

ModuleNotFoundError: No module named 'imageai'